In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Using `ipyvolume`

In [3]:
import ipyvolume as ipv

## Examples

In [23]:

# 
# import pandas as pd

# V = np.zeros((128,128,128)) # our 3d array
# # outer box
# V[30:-30,30:-30,30:-30] = 0.75
# V[35:-35,35:-35,35:-35] = 0.0
# # inner box
# V[50:-50,50:-50,50:-50] = 0.25
# V[55:-55,55:-55,55:-55] = 0.0
# ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)

In [24]:
# ipv.pylab.clear()
# ipv.examples.gaussian(N=1000, draw=True, show=True, seed=42, color=None, marker='sphere')

In [25]:
# ipv.pylab.clear()
# V = ipv.examples.xyz(shape=128, limits=[-3, 3], spherical=True, sparse=True, centers=True)
# #ipv.quickvolshow(V, opacity=0.03, level_width=0.1, data_min=0, data_max=1)

In [26]:
# ipyvolume.pylab.clear()
# ipyvolume.examples.head(draw=True, show=True, max_shape=256)

In [27]:
# ipv.pylab.clear()
# ipv.examples.ball(rmax=3, rmin=0, shape=128, limits=[-4, 4], draw=True, show=True,)

In [28]:
# ipv.examples.example_ylm(m=0, n=2, shape=128, limits=[-4, 4], draw=True, show=True,)

## functions I need

In [4]:
def xyz(shape=128, limits=[-3, 3], coordinate='cartesian', sparse=True, centers=False):
    """
    this function is adapted from ipyvolume source code for
    different coordinate systems in 3D.
    """
    
    # making the shape
    dim = 3
    try:
        shape[0]
    except:
        shape = [shape] * dim
    
    # setting the limits (better to change it such that we get it as arg)
    try:
        limits[0][0]
    except:
        limits = [limits] * dim
    
    # makes the cube of coordinates
    if centers:
        #print([(   vmin+(vmax-vmin)/float(N)/2, vmax-(vmax-vmin)/float(N)/4, (vmax-vmin)/float(N)) for (vmin, vmax), N in zip(limits, shape)])
        v = [slice(vmin+(vmax-vmin)/float(N)/2, vmax-(vmax-vmin)/float(N)/4, (vmax-vmin)/float(N)) for (vmin, vmax), N in zip(limits, shape)]
    else:
        v = [slice(vmin, vmax+(vmax-vmin)/float(N)/2, (vmax-vmin)/float(N-1)) for (vmin, vmax), N in zip(limits, shape)]
    
    if sparse:
        x, y, z = np.ogrid.__getitem__(v)
    else:
        x, y, z = np.mgrid.__getitem__(v)
    
    
    if coordinate == 'spherical':
        r = np.linalg.norm([x, y, z])
        theta = np.arctan2(y, x)
        phi = np.arccos(z / r)
        return x, y, z, r, theta, phi
    elif coordinate == 'cylindrical':
        r = np.linalg.norm([x, y, 0*z])
        phi = np.arctan2(y, x)
        return x, y, z, r, phi, np.ones_like(r)*z
    else:
        return x, y, z
    
    
def cylinder(R, thickness=1.5, shape=128, limits=[-4, 4], draw=True, show=True, **kwargs):
    """Show a cylinder"""
    import ipyvolume.pylab as p3
    #p3.style.use('bmh')
    #p3.style.axes_off()
    _, _, _, r, phi, z = xyz(shape=shape, limits=limits, coordinate='cylindrical')
    data = r * 0
    data[(r < R+thickness) & (r >= R)] = 10
    if "data_min" not in kwargs:
        kwargs["data_min"] = 0
    if "data_max" not in kwargs:
        kwargs["data_max"] = 1
    data  = data.T
    if draw:
        vol = p3.volshow(data=data, **kwargs)
        if show:
            p3.show()
        return vol
    else:
        return data

## Making a cylinder

In [5]:
_,_,_,r,phi, z = xyz(coordinate='cylindrical')

In [6]:
ipv.clear()
cylinder(3,thickness=0.5,limits=[-10,10])

/home/arash/miniconda3/envs/dril/lib/python3.7/site-packages/ipyvolume/serialize.py:81: RuntimeWarning:

invalid value encountered in true_divide



Volume(data=array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0.,…

In [7]:
ipv.clear()
ipv.pylab.plot_isosurface(r,1, extent=[[0,128],[0,128],[0,128]])
#ipv.plot(np.array([0,168]),np.array([0,0]),np.array([0,30]), color='blue',limits=[-3,3])
ipv.show()

## Notes 
This is not good:
1. the plot_isosurface, cannot dynamically change teh radius.
2. The vector that I need to plot, would change by chaning the level of surface.
3. the non-iso_surface one sucks!

# Using `Plotly`

In [8]:
import plotly.plotly as py
import plotly.graph_objs as go

In [9]:

phi = np.linspace(0, 2 * np.pi, 360)
z = np.linspace(0, 10, 100)
phi_grid, z_grid = np.meshgrid(phi, z)


# the vector end points
X = [0, 3]
Y = [0, 4]
Z = [0, 2]

# the cylinder
r = np.sqrt(X[1]**2 + Y[1]**2)
x = r * np.cos(phi_grid)
y = r * np.sin(phi_grid)
surface = go.Surface(x=x, y=y, z=z_grid, opacity=0.5, colorscale='Blues',showscale = False)


# projection on xy-plane
xy_proj = go.Scatter3d(x=X, y=Y, z=[0,0],
                       line=dict(color='rgb(128,0,0)',width=5, dash ='dash'),
                       marker=dict(size=0) )

# projection on x-axis
x_proj = go.Scatter3d(x=[0,X[1]], y=[0,0], z=[0,0],
                       line=dict(color='rgb(0,128,0)',width=5, dash ='dash'),
                       marker=dict(size=0), )

# projection on y-axis
y_proj = go.Scatter3d(x=[0,0], y=[0,Y[1]], z=[0,0],
                       line=dict(color='rgb(0,128,0)',width=5, dash ='dash'),
                       marker=dict(size=0) )

# projection on z-axis
z_proj = go.Scatter3d(x=[0,0], y=[0,0], z=[0,Z[1]],
                       line=dict(color='rgb(0,128,0)',width=5, dash ='dash'),
                       marker=dict(size=0) )


# projection on z-axis
xy2x_proj = go.Scatter3d(x=[X[1],X[1]], y=[Y[1],0], z=[0,0],mode = 'lines',
                       line=dict(color='rgb(0,0,0)',width=5, dash ='dash') )

xy2y_proj = go.Scatter3d(x=[X[1],0], y=[Y[1],Y[1]], z=[0,0],mode = 'lines',
                       line=dict(color='rgb(0,0,0)',width=5, dash ='dash') )

vec2z_proj = go.Scatter3d(x=[X[1],0], y=[Y[1],0], z=[Z[1],Z[1]],mode = 'lines',
                       line=dict(color='rgb(0,0,0)',width=5, dash ='dash') )

vec2xy_proj = go.Scatter3d(x=[X[1],X[1]], y=[Y[1],Y[1]], z=[Z[1],0],mode = 'lines',
                       line=dict(color='rgb(256,0,0)',width=5, dash ='dash') )

Vector = go.Scatter3d(x=X, y=Y, z=Z,
                      marker=dict(size=6, color='rgb(0,0,0)'),
                      line=dict(color='rgb(0,0,0)', width=5))



layout_axis_config = dict(gridcolor='rgb(255, 255, 255)',
                            zerolinecolor='rgb(255, 255, 255)',
                            backgroundcolor='rgb(230, 230,230)',
                            zerolinewidth=4,
                            showbackground=True,
                            )
layout = go.Layout(title='Cylindrical Coordinate',
                   scene=dict(xaxis= layout_axis_config,
                              yaxis= layout_axis_config,
                              zaxis= layout_axis_config),
                   showlegend=False)

fig = go.Figure(data=[surface, Vector, xy_proj, x_proj, y_proj, z_proj, 
                      xy2x_proj, xy2y_proj, vec2z_proj, vec2xy_proj], 
                layout=layout)
py.iplot(fig, filename='Parametric_plot')

/home/arash/miniconda3/envs/dril/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

